<a href="https://colab.research.google.com/github/valerianXZ/Project/blob/main/WebScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pygsheets selenium
import pandas as pd
import requests
import pygsheets
import json
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
!apt-get install -y chromium-browser
!apt install chromium-chromedriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd
import pygsheets

In [ ]:
from google.colab import files
uploaded=files.upload()

In [24]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
)


service = Service(executable_path="/usr/lib/chromium-browser/chromedriver")
driver = webdriver.Chrome( options=options)


In [25]:
# Get data from Google Search Engine
query = "Machine learning skills needed 2025"
search_url = f"https://www.google.com/search?q={query}&hl=zh-TW&gl=tw"

print("Opening Google Search...")
driver.get(search_url)
time.sleep(5)

urls=[]
search_results = driver.find_elements(By.CSS_SELECTOR, "div.tF2Cxc")

print(f"Found {len(search_results)} search results")


for result in search_results:
    try:
        title = result.find_element(By.TAG_NAME, "h3").text
        link_element = result.find_element(By.TAG_NAME, "a")
        link = link_element.get_attribute("href")

        print(f"Title: {title}, URL: {link}")
        urls.append((title, link))
    except Exception as e:
        print(f"Error extracting title/link: {e}")

# If no URLs were found, stop execution
if not urls:
    print("No search results found. Exiting...")
    driver.quit()
    exit()

data = []
for title, url in urls:
    try:
        print(f"Visiting: {url}")
        driver.get(url)
        time.sleep(5)


        try:
            meta_description = driver.find_element(By.NAME, "description").get_attribute("content")
        except:
            meta_description = "No meta description found"

        # Get the  Content
        paragraphs = driver.find_elements(By.TAG_NAME, "p")
        content = "\n".join([p.text for p in paragraphs])  # Limit to first 10 paragraphs for faster processing

        data.append([title, meta_description, content, url])
    except Exception as e:
        print(f"Error extracting content from {url}: {e}")

driver.quit()

Opening Google Search...
Found 14 search results
Title: 12 Machine Learning Skills to Power Your Career in 2025, URL: https://careerfoundry.com/en/blog/data-analytics/machine-learning-skills/
Title: The ML Engineer: 7 Essential Skills to Master in 2025, URL: https://medium.com/@piyushkashyap045/the-ml-engineer-7-essential-skills-to-master-in-2025-25135d48640f
Title: Machine Learning Engineer Skills, URL: https://www.tealhq.com/skills/machine-learning-engineer
Title: Future-Proof Your Machine Learning Career in 2025, URL: https://machinelearningmastery.com/future-proof-machine-learning-career-2025/
Title: The 2025 Machine Learning Job Market: Trends, Salaries, ..., URL: https://www.linkedin.com/pulse/2025-machine-learning-job-market-trends-salaries-skills-shields-16bye
Title: AI & Machine Learning Careers in 2025: Skills and Growth ..., URL: https://medium.com/@vaishnaviyada/ai-machine-learning-careers-in-2025-skills-and-growth-opportunities-fc284c1bf09d
Title: Must-Have Machine Learnin

In [37]:
#Set as dataFrame and import to google sheet
from tabulate import tabulate
df = pd.DataFrame(data, columns=["Title", "Meta_Description", "Content", "URL"])
df["Content"]=df["Content"].str.replace(r'\n+', ' ',regex=True)
display(df.head(10))

gc = pygsheets.authorize(service_file="practice-449103-1a3eb945fca7.json")
sh = gc.open("DataSkills")
wks = sh.sheet1
wks.set_dataframe(df, (1, 1))

print("Data successfully uploaded to Google Sheets!")


,Title,Meta_Description,Content,URL
0,12 Machine Learning Skills to Power Your Caree...,Data expert Austin takes you through 12 of the...,More Categories So you’ve been hearing about ...,https://careerfoundry.com/en/blog/data-analyti...
1,The ML Engineer: 7 Essential Skills to Master ...,The role of a Machine Learning (ML) Engineer h...,Sign up Sign in Piyush Kashyap Follow 10 The ...,https://medium.com/@piyushkashyap045/the-ml-en...
2,Machine Learning Engineer Skills,Learn which skills will be most essential for ...,Learn about the most important skills for Mach...,https://www.tealhq.com/skills/machine-learning...
3,Future-Proof Your Machine Learning Career in 2025,No meta description found,Making developers awesome at machine learning...,https://machinelearningmastery.com/future-proo...
4,"The 2025 Machine Learning Job Market: Trends, ...","WSDA News | January 31, 2025 As we step into a...",LINKEDIN LinkedIn is better on the app Don’t ...,https://www.linkedin.com/pulse/2025-machine-le...
5,AI & Machine Learning Careers in 2025: Skills ...,Artificial Intelligence (AI) and Machine Learn...,Sign up Sign in Vaishnavi Yada (Amira) Follow...,https://medium.com/@vaishnaviyada/ai-machine-l...
6,Must-Have Machine Learning Skills in 2025,"In this guide, we will cover the must-have Mac...",Blog Artificial Intelligence and Machine Learn...,https://www.guvi.in/blog/top-skills-to-become-...
7,The 14 Essential AI Engineer Skills You Need t...,AI engineer skills are now in demand. Learn al...,Last chance! 50% off unlimited learning Sale e...,https://www.datacamp.com/blog/essential-ai-eng...
8,5 Top Skills for AI Enthusiasts in 2025,Explore the top AI skills for 2025 to stay ahe...,Artificial Intelligence (AI) is quickly becomi...,https://www.elearningcollege.com/elearning-blo...
9,Top Data Science Skills You Need in 2025 for C...,Discover the top 20+ data science skills you n...,Having the latest abilities is essential for ...,https://www.simplilearn.com/what-skills-do-i-n...


Data successfully uploaded to Google Sheets!
